# End of week 1 exercise

To demonstrate your familiarity with OpenAI API, and also Ollama, build a tool that takes a technical question,  
and responds with an explanation. This is a tool that you will be able to use yourself during the course!

In [4]:
# imports
import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [5]:
# constants

MODEL_GPT = 'gpt-4o-mini'
MODEL_LLAMA = 'llama3.2'

OLLAMA_API = "http://localhost:11434/api/chat"



In [6]:
# set up environment

# Load environment variables in a file called .env
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

# Check the key

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")



def messages_question(question):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

system_prompt = "You are a technical assistant that takes questions \
and provides a detailed explaination with examples if possible \
Respond in markdown."

openai = OpenAI()

API key found and looks good so far!


In [7]:
# here is the question; type over this to ask something new

question = """
Please explain what this code does and why:
yield from {book.get("author") for book in books if book.get("author")}
"""

In [9]:
# Get gpt-4o-mini to answer, with streaming
def stream_question(question):
    stream = openai.chat.completions.create(
        model=MODEL_GPT,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": question}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

stream_question(question)

Certainly! Let's break down the provided code step-by-step to understand what it does and why it is structured that way.

### Code Breakdown

python
yield from {book.get("author") for book in books if book.get("author")}


1. **`{...}` (Set Comprehension)**:
   - The curly braces `{}` indicate that we are creating a set comprehension. A set is a data structure that holds unordered collections of unique elements. In this case, we're creating a set of authors.

2. **`for book in books`**:
   - This part iterates over an iterable called `books`. Here, `books` is expected to be a collection (like a list) of dictionaries, where each dictionary likely represents information about a book.

3. **`book.get("author")`**:
   - The `get` method is used to retrieve the value associated with the key `"author"` from the dictionary `book`. If the key does not exist, it will return `None` instead of throwing an error.

4. **`if book.get("author")`**:
   - This conditional checks whether the value retrieved by `get("author")` is truthy. In Python, empty strings and `None` are considered falsy. Hence, this condition ensures that only books with a valid author will be included in the set comprehension.

5. **`yield from`**:
   - `yield from` is used in Python generators. It allows you to yield all values from an iterable (in this case, the set of authors) one by one. This means that the generator will produce each author without needing to loop through the elements manually.

### Putting It All Together

The entire line of code is a generator expression that extracts unique authors from a list of book dictionaries. Each author is obtained through the `get` method, and only non-empty values are included. The `yield from` allows a generator function to yield each unique author one at a time.

### Example

Here’s an example to illustrate what this code does.

#### Example `books` List

python
books = [
    {"title": "Book 1", "author": "Author A"},
    {"title": "Book 2", "author": "Author B"},
    {"title": "Book 3"},
    {"title": "Book 4", "author": "Author A"},
    {"title": "Book 5", "author": None},
]


#### Using the Code in a Generator Function

python
def get_unique_authors(books):
    yield from {book.get("author") for book in books if book.get("author")}


#### Iterating Through the Generator

python
for author in get_unique_authors(books):
    print(author)


#### Output


Author A
Author B


### Summary

- The code efficiently extracts unique authors from a collection of book dictionaries.
- It utilizes a set comprehension to avoid duplicates and includes only valid author entries.
- The use of `yield from` allows the function to yield these unique authors one by one to the caller. 

This approach is concise and efficient for working with potentially large datasets, ensuring clarity by only returning valid author names.

In [10]:
# Get Llama 3.2 to answer

def stream_llama_question(question):
    ollama_via_openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')

    stream = ollama_via_openai.chat.completions.create(
        model=MODEL_LLAMA,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": question}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

stream_question(question)



Certainly! Let's break down this code snippet step by step to understand what it does and why:

python
yield from {book.get("author") for book in books if book.get("author")}


### Components of the Code

1. **`yield from`**: 
   - This is a keyword used in generator functions in Python. When you use `yield from <iterable>`, it allows you to yield all values from the iterable directly. It's a way of delegating part of a generator's operations to another generator.

2. **Set Comprehension**:
   - The expression `{book.get("author") for book in books if book.get("author")}` is a set comprehension. 
   - A set comprehension constructs a set by iterating through an iterable (in this case, `books`) and applying an expression (the `book.get("author")` part) to each item, but only includes items that meet a condition (`if book.get("author")`).

3. **Accessing Dictionary Values**:
   - The method `book.get("author")` is used to retrieve the value associated with the key `"author"` from the dictionary `book`. 
   - If the `"author"` key does not exist, it returns `None` instead of raising a KeyError.

4. **Filtering**:
   - The condition `if book.get("author")` ensures that only books that have a non-None author are included in the resulting set. This helps in filtering out any entries that either lack the author key altogether or have a `None` value for the author.

### What This Code Does

- It iterates over a collection of `books` (which is assumed to be a list of dictionaries, where each dictionary represents a book).
- For each `book`, it fetches the `author` using `book.get("author")`.
- It retains only those authors that are not `None` (i.e., books that actually have an author).
- The resulting authors are captured in a set, which inherently eliminates any duplicate authors (since sets don’t allow duplicate entries).
- Finally, `yield from` yields each author found in the set one by one to any caller of the generator function.

### Example

Consider the following example:

python
books = [
    {"title": "Book 1", "author": "Author A"},
    {"title": "Book 2", "author": None},
    {"title": "Book 3", "author": "Author B"},
    {"title": "Book 4"},  # No author key
    {"title": "Book 5", "author": "Author A"}
]

def author_generator(books):
    yield from {book.get("author") for book in books if book.get("author")}

# Using the generator
for author in author_generator(books):
    print(author)


### Output


Author A
Author B


### Summary

- The given code is a concise way of constructing a unique set of authors from a list of books with potential missing or None values for authors. The use of `yield from` and set comprehension makes it both efficient and pythonic.